<a href="https://colab.research.google.com/github/rbaner21/technosignature-pipeline-v2/blob/main/notebooks/1b_tess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: ensure inputs/tess exists & load the top200 list
import os, pandas as pd
os.makedirs("inputs/tess", exist_ok=True)

tt = pd.read_csv("inputs/tess_targets.csv")
print(f"✅ Will run TESS on {len(tt)} targets")


In [ ]:
# Cell 2: loop over the 200 TICs & flag transit anomalies
import numpy as np, pandas as pd
import lightkurve as lk
from astropy.timeseries import BoxLeastSquares

tt = pd.read_csv("inputs/tess_targets.csv")
flags = []

for tic in tt["tic_id"].dropna().unique():
    try:
        lc = lk.search_lightcurve(f"TIC {int(tic)}", mission="TESS").download()
        lc_flat = lc.flatten(window_length=401)
        bls = BoxLeastSquares(lc_flat.time, lc_flat.flux)
        periods = np.linspace(0.5,10,5000)
        power = bls.power(periods,0.1).power
        depth = bls.depth.max()
        snr   = power.max() / np.std(lc_flat.flux)
        flags.append({
            "tic_id":     tic,
            "tess_flag":  bool((depth>0.05) & (snr>20)),
            "tess_period":float(periods[np.argmax(power)]),
            "tess_snr":   float(snr)
        })
    except Exception as e:
        print(f"TIC {tic} error: {e}")

pd.DataFrame(flags).to_csv("inputs/tess_flags.csv", index=False)
print(f"✅ Phase 2 TESS done ({len(flags)} flags)")
